In [3]:
# Install minimal dependencies (`torch`, `transformers`, `timm`, `tokenizers`, ...)
# > pip install -r https://raw.githubusercontent.com/openvla/openvla/main/requirements-min.txt
from transformers import AutoModelForVision2Seq, AutoProcessor
from PIL import Image

import torch

# Load Processor & VLA
processor = AutoProcessor.from_pretrained("openvla/openvla-7b", trust_remote_code=True)
vla = AutoModelForVision2Seq.from_pretrained(
    "openvla/openvla-7b", 
    attn_implementation="flash_attention_2",  # [Optional] Requires `flash_attn`
    torch_dtype=torch.bfloat16, 
    low_cpu_mem_usage=True, 
    trust_remote_code=True
).to("cuda:0")

image_path = '/home/xilun/openvla/test_image.png'
# load_image 
def get_from_camera(image_path: str) -> Image.Image:
    return Image.open(image_path)
# Grab image input & format prompt
image: Image.Image = get_from_camera(image_path)
prompt = "In: What action should the robot take to {Lift the blue cube}?\nOut:"

# Predict Action (7-DoF; un-normalize for BridgeData V2)
inputs = processor(prompt, image).to("cuda:0", dtype=torch.bfloat16)
action = vla.predict_action(**inputs, unnorm_key="bridge_orig", do_sample=False)

print (action) 

# # Execute...
# robot.act(action, ...)

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  4.86it/s]


[ 0.00471205 -0.00171916 -0.00569757  0.00305409 -0.01819708  0.00799228
  0.99607843]
